In [1]:
import numpy as np
import shutil
import os
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.xception import Xception, preprocess_input, decode_predictions

Using TensorFlow backend.


In [12]:
def create_generators(train_data_dir, validation_data_dir):
    # Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.

    transformation_ratio = .05  # how aggressive will be the data augmentation/transformation

    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                        batch_size=BATCH_SIZE,
                                                        class_mode='categorical')

    validation_datagen = ImageDataGenerator(rescale=1. / 255)
    
    validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                                  batch_size=BATCH_SIZE,
                                                                  class_mode='categorical')
    return train_generator, validation_generator


def create_model(num_classes):
        base_model = Xception(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights='imagenet', include_top=False)

        # Top Model Block
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        predictions = Dense(num_classes, activation='softmax')(x)

        # add your top layer block to your base model
        model = Model(base_model.input, predictions)
        
        for layer in model.layers[:-10]:
            layer.trainable = False
        
        model.compile(optimizer='nadam',loss='categorical_crossentropy', metrics=['accuracy'])
        return model


def train(train_generator, validation_generator, model):
    es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    model.fit_generator(train_generator,
                        callbacks=[es],
                        epochs=1,
                        validation_data=validation_generator,
                        steps_per_epoch=4,
                        validation_steps=2,
                        verbose=1)


# makes the prediction of the file path image passed as parameter 
def predict(file, model, to_class):
    im = load_img(file, target_size=(IMG_WIDTH, IMG_HEIGHT))
    x = img_to_array(im)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    index = model.predict(x).argmax()
    return to_class[index]

In [13]:
DIR = "train_model/"
IMG_WIDTH, IMG_HEIGHT = 299, 299 # set this according to keras documentation, each model has its own size
BATCH_SIZE = 200 # decrease this if your computer explodes

In [14]:
train_generator, validation_generator = create_generators(DIR + "labelled", DIR + "validation")

total_classes = len(train_generator.class_indices) # necesary to build the last softmax layer
to_class = {v:k for k,v in train_generator.class_indices.items()} # usefull when model returns prediction
to_class

Found 3423 images belonging to 6 classes.
Found 745 images belonging to 6 classes.


{0: 'bathroom',
 1: 'bedroom',
 2: 'exterior',
 3: 'kitchen',
 4: 'living',
 5: 'plan'}

In [15]:
m = create_model(total_classes)

In [16]:
train(train_generator, validation_generator, m)

Epoch 1/1
4/4 [==============================] - 121s 30s/step - loss: 0.9373 - accuracy: 0.6375 - val_loss: 0.4769 - val_accuracy: 0.8450


In [17]:
train(train_generator, validation_generator, m)

Epoch 1/1
4/4 [==============================] - 110s 27s/step - loss: 0.3323 - accuracy: 0.8888 - val_loss: 0.5673 - val_accuracy: 0.8025


In [18]:
train(train_generator, validation_generator, m)

Epoch 1/1
4/4 [==============================] - 110s 27s/step - loss: 0.2236 - accuracy: 0.9162 - val_loss: 0.4879 - val_accuracy: 0.8800


In [ ]:
# execute this when you want to save the model
m.save('room_classifier')

# execute this when you want to load the model
#from keras.models import load_model
#m = load_model('room_classifier')

In [20]:
multiprocessing.cpu_count()

16

In [ ]:
image_dir = os.listdir(path)

In [ ]:
#process_images('images')
def process_images(fld):
    path = 'images'
    dest_path = 'images_classed'
    for img in os.listdir(path+"/"+fld):
        try:
            file = img.split('_')[0]
            source_dir = path+"/" +str(file)+"/"+str(img)
            room_class = predict(source_dir, m, to_class)
            dest_source = dest_path+"/"+room_class
            shutil.copy(src=source_dir,dst=dest_source)
        except:
            print('Failed: ' + source_dir)

path = 'images'
image_dir = os.listdir(path)
num_cores = multiprocessing.cpu_count()

if __name__ == '__main__':
    processed_list = Parallel(n_jobs=num_cores-1)(delayed(process_images)(i) for i in tqdm(image_dir))
    
    



  0%|          | 0/23052 [00:00<?, ?it/s]

  0%|          | 15/23052 [00:03<1:28:15,  4.35it/s]

  0%|          | 30/23052 [00:19<3:08:05,  2.04it/s]

  0%|          | 45/23052 [00:55<6:42:11,  1.05s/it]

  0%|          | 60/23052 [01:30<9:12:42,  1.44s/it]

  0%|          | 75/23052 [02:06<10:59:01,  1.72s/it]

  0%|          | 90/23052 [02:42<12:15:15,  1.92s/it]

  0%|          | 105/23052 [03:27<14:24:25,  2.26s/it]

  1%|          | 120/23052 [03:54<13:29:56,  2.12s/it]

  1%|          | 135/23052 [04:28<13:44:53,  2.16s/it]

  1%|          | 150/23052 [05:09<14:51:23,  2.34s/it]

  1%|          | 165/23052 [05:42<14:31:24,  2.28s/it]

  1%|          | 180/23052 [06:17<14:40:01,  2.31s/it]

  1%|          | 195/23052 [07:08<16:43:14,  2.63s/it]

  1%|          | 210/23052 [07:45<16:22:22,  2.58s/it]

  1%|          | 225/23052 [08:21<16:03:01,  2.53s/it]

  1%|          | 240/23052 [09:09<17:21:34,  2.74s/it]

  1%|          | 255/23052 [09:45<16:39:52,  2.63s/it]

  1%|        

In [8]:
path = 'images'
image_dir = os.listdir(path)


In [9]:
for fld in image_dir:
    for img in os.listdir(path+"/"+fld):
        file = img.split('_')[0]
        source_dir = path+"/" +str(file)+"/"+str(img)
        room_class = predict(source_dir, m, to_class)
        print(room_class)


exterior
exterior
kitchen
kitchen
bathroom
kitchen
kitchen
bedroom
kitchen
exterior
kitchen
kitchen
kitchen
bedroom
exterior
kitchen
kitchen
bedroom
bedroom
bedroom
bedroom
kitchen
bedroom
bedroom
exterior
exterior
kitchen
bedroom
exterior
bathroom
bedroom
bathroom
bedroom
exterior
bedroom
kitchen
bathroom
exterior
kitchen
kitchen
exterior
kitchen
kitchen
kitchen
bedroom
exterior
kitchen
kitchen
bedroom
bedroom
bedroom
bathroom
bedroom
kitchen
exterior
bathroom
kitchen
kitchen
bedroom
kitchen
kitchen
kitchen
bedroom
bedroom
exterior
bedroom
bathroom
exterior
kitchen
kitchen
bedroom
exterior
bathroom
exterior
exterior
bedroom
kitchen
kitchen
kitchen
kitchen
bedroom
exterior
kitchen
bedroom
bathroom
exterior
bathroom
bedroom
exterior
bedroom
exterior
exterior
exterior
exterior
exterior
kitchen
bedroom
kitchen
exterior
kitchen
kitchen
exterior
bedroom
bedroom
bedroom
kitchen
kitchen
bathroom
bedroom
exterior
kitchen
kitchen
bedroom
bedroom
kitchen
bathroom
bedroom
kitchen
exterior
bathroo

KeyboardInterrupt: 